In [ ]:
import pandas as pd

# https://portal.gdc.cancer.gov/
# tar -xvzf C:\Users\Ameno\Downloads\clinical.cases_selection.2022-03-29.tar.gz -C C:\Users\Ameno\Downloads\HNSC_clinical

In [ ]:
# df = pd.read_csv('sample_metadata.txt', delimiter = "\t")
g = pd.read_csv('Data/HNSC/tcga_scaled01_rnaseq.tsv', delimiter = "\t", index_col=0)
# b.head()
# b.head()
# df.head()
# list(df.columns)

In [ ]:
# Explore SNF data
g_snf = pd.read_csv("Data\COAD\COLON_Gene_Expression.txt", delimiter = "\t", index_col=0).T
print(g_snf.head(2))
# print(g.head(2))

In [ ]:
x_cancer_type = ["COAD", "ESCA", "HNSC", "READ", "STAD"]

#stad has the most normal samples 
cancer = x_cancer_type[4]
# cancer = "HNSC"
# cancer = "ESCA"
# cancer = "READ"

b = pd.read_csv(f'Data/{cancer}/Microbial/bacteria.sample.relabund.phylum.txt', delimiter = "\t", index_col=0).T

In [ ]:
g.head()
# b.head()

g.iloc[:,3]
g_samples = list(g.index.values)
b_samples = list(b.index.values)
set(g_samples).intersection(b_samples)
b_samples
# print(g.columns)
# print(g.describe())

find = "TCGA-CR-6491"
# find = "TCGA-CV-7090"

# TCGA-ZX-AA5X-01 does not occur, and has duplicates. Not true, it has it. It just doesn't have some of the elevens ( other elevens it does ), and take care of the b ones

# HNSC 

def findOverlap(a_samples, b_samples, ignoreVial=False):
    overlapping_count = 0
    normal_overlapping_count = 0
    missing = []
    control_count = 0
    b_control_count = 0
    
    print(f"checking a ({len(a_samples)}) vs. b ({len(b_samples)})")
    for find in a_samples:
        occurrences = 0
        for s in b_samples:
            if ignoreVial and find[-1]=="A":
                find = find[:-1]
            if find in s:
                occurrences+=1
        if occurrences<1:
            # print(find, " does not have any occurrences")
            missing.append(find)
        else:
            overlapping_count += 1
        if ((ignoreVial and find[-2] == "1") or (not ignoreVial and find[-3] == "1")):
            control_count += 1
            if occurrences > 0:
                normal_overlapping_count += 1
    
    for find in b_samples:
        if ((ignoreVial and find[-2] == "1") or (not ignoreVial and find[-3] == "1")):
            b_control_count += 1
            
    print("no occurrences:", missing)
    print("overlapping count: ", overlapping_count, "/", len(a_samples))
    print("normal count a: ", control_count, "/", len(a_samples))
    print("normal count b: ", b_control_count, "/", len(b_samples))
    print("normal overlapping count: ", normal_overlapping_count, "/", control_count)
    return missing
rna_stad_missing = findOverlap(b_samples, g_samples, ignoreVial=True)
l1=[]
for i in b:
    if i not in l1:
        l1.append(i)
    else:
        print(i,end=' ')


# tumor normal and control 
import re
for s in g_samples:
    if "4183" in s:
    # if re.search(r"[1].$",s):
    # if "-21" in s:
        
        print(s)

In [ ]:
for c in x_cancer_type:
    cancer_microbiome = pd.read_csv(f'Data/{c}/Microbial/bacteria.sample.relabund.phylum.txt', delimiter = "\t", index_col=0).T
    cancer_microbiome["tumor"] = cancer_microbiome.apply(lambda row: 1 if re.search(r"[0]..$",row.name) else 0, axis=1)
    # cancer_microbiome["cancer"] = c
    print(f"{c}:", cancer_microbiome.tumor.value_counts())
    # print(cancer_microbiome.head(2))
    # print(list(cancer_microbiome.columns))

In [ ]:
x_type = ["cnv", "methylation", "rnaseq"]
# x_cancer_type = ["COAD", "READ", "STAD"]
xe_rna_stad_samples = pd.read_csv(rf"Data\{cancer}\xenos_{x_type[2]}_samples.txt", names=["sample"], header=None)
xe_rna_stad_samples.head(2)

print(f"Number of xenos samples:", xe_rna_stad_samples.shape[0])
xe_rna_stad_missing = findOverlap(b_samples, xe_rna_stad_samples["sample"].tolist())
xe_rna_stad_missing = map(lambda x: x[:-1], xe_rna_stad_missing)
with_xe_difference = set(xe_rna_stad_missing).difference(set(rna_stad_missing))
from_xe_difference = set(rna_stad_missing).difference(set(xe_rna_stad_missing))

print("x-tc:", with_xe_difference, " tc-x:", from_xe_difference)
# findOverlap(b_samples, g_snf_samples_mapped)

In [ ]:
akash_methylation_stad_samples = pd.read_csv(rf"Data\{cancer}\TCGA_STAD_Samples.csv", names=["sample"], header=None, skiprows=1)
akash_methylation_stad_samples = list(map(lambda x: x.replace(".", "-"), akash_methylation_stad_samples["sample"].tolist()))
akash_methylation_stad_missing = findOverlap(b_samples, akash_methylation_stad_samples, ignoreVial=True)
akash_methylation_stad_samples
[print(x) for x in akash_methylation_stad_samples if "11" in x]

In [ ]:
g2 = pd.read_csv('Data\pancan_scaled_rnaseq.tsv\pancan_scaled_rnaseq.tsv', delimiter = "\t", index_col=0)

In [ ]:
g2_samples = list(g2.index.values)
# len(g_samples)
same = set(g2_samples).intersection(g_samples)
len(same)

In [ ]:
# g.rename(columns={0:"name"}, inplace=True)
g.index.name = "name"
print(g.head(2))
b.index
b.head(2)

In [ ]:
b.rename(index= lambda s: s[:-1] if s[-1]=="A"else s, inplace=True)
multi = b.join(g, how="inner")
# multi.loc["TCGA-CV-5970-11"]
multi.index.name = "patient"
# multi["patient"]
# multi["tumor"] = multi.apply(lambda row: 1 if re.search(r"[0].$",row.name) else 0)
multi["tumor"] = multi.apply(lambda row: 1 if re.search(r"[0].$",row.name) else 0, axis=1)
# multi.apply(lambda row: print(row.name))
# multi.to_csv(f"Data/{cancer}/gene_biome_{cancer}.csv")
# pd.merge(b, g, how="inner", left_on="name")
# g.merge(b, how="inner", on="name")

In [ ]:
all_cancer_genus = pd.read_csv(f'Data/TCMA/all_cancers_genus.csv', index_col=0)
all_cancer_genus.rename(index= lambda s: s[:-1] if s[-1]=="A"else s, inplace=True)

genus_ge = all_cancer_genus.join(g, how="inner")
genus_ge.index.name = "patient"
genus_ge["tumor"] = genus_ge.apply(lambda row: 1 if re.search(r"[0].$",row.name) else 0, axis=1)
genus_ge.to_csv(f"Data/TCMA/all_cancers_genus_ge(aakash).csv")
print(genus_ge.head(1))
